# Работа с текстовыми данными

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=True,random_state=7)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True,random_state=7)#

Одним из направлений машинного обучения является работа с текстами и извлечение полезной информации из текстов. Чтобы алгоритмы машинного обучения могли работать с текстами, необходимо перевести тексты в наборы чисел. Для этого применяют различные алгоритмы векторизации текстов.

Будем рассматривать данные, которые называются “Twenty Newsgroups”. Вот его официальное описание, взятое с [сайта](http://qwone.com/~jason/20Newsgroups/): 

Данные «The 20 Newsgroups» — это коллекция примерно из 20000 новостных документов, разделенная (приблизительно) равномерно между 20 различными категориями. Насколько нам известно, изначально она собиралась Кеном Ленгом (Ken Lang), возможно, для его работы «Newsweeder: Learning to filter netnews» («Новостной обозреватель: учимся фильтровать новости из сети»), хотя он явно не заявлял об этом. Коллекция «The 20 newsgroups» стала популярным набором данных для экспериментов с техниками машинного обучения для текстовых приложений, таких как классификация текста или его кластеризация.

1. Импортируем данные 4 категорий с помощью [fetch_20newsgroups](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html).

In [2]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=True,random_state=7)#

In [3]:
twenty_train.target

array([ 5,  1,  6, ..., 15,  4,  8])

In [4]:
twenty_train.target_names # значения целевой переменной 

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [5]:
print(len(twenty_train.data)) #количество документов в обучающей выборке

11314


In [6]:
print(twenty_train.data[250]) # типичный документ 

From: gld@cunixb.cc.columbia.edu (Gary L Dare)
Subject: Re: Z Magazine: Health Care Reform (March 93)
Nntp-Posting-Host: cunixb.cc.columbia.edu
Reply-To: gld@cunixb.cc.columbia.edu (Gary L Dare)
Organization: PhDs In The Hall
Lines: 95

{Sorry, Harel et al, but our doctors and most hospitals are still
private in Canada as well as in much of Western Europe.}

harelb@math.cornell.edu (Harel Barzilai - Grad - Brown) writes:
>		 =================================== 
>		 H e a l t h   C a r e   R e f o r m 
>			 By Camille Colatosti 
>		 Z magazine (see bottom), March 1993 
>		 =================================== 
>...
>The single-payer model, sometimes called national health insurance, 
>eliminates private insurance companies and removes health care from 
>employment. The government provides free health care to all U.S. 
>residents. And there are no out-of-pocket costs. 

Wrong.  In better EC countries that use pure (but public) health
insurance (like we use in Canada) rather than self-encl

In [7]:
y=twenty_train.target[250]
print(y)
print(twenty_train.target_names[y]) # и категория документа 

18
talk.politics.misc


In [8]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t]) # категории первых 10 документов

comp.windows.x
comp.graphics
misc.forsale
talk.politics.guns
talk.politics.guns
rec.autos
comp.windows.x
sci.crypt
comp.sys.mac.hardware
talk.politics.mideast


#### 1. А что делает вот такая функция? 

In [9]:
print(np.bincount(twenty_train.target))

[480 584 591 590 578 593 585 594 598 597 600 595 591 594 593 599 546 564
 465 377]


## Мешок слов

* составим словарь слов, появляющихся в документах обучающей выборки; 

* каждому слову припишем уникальный целочисленный индекс $j$; 

* для каждого документа вычислим количество употреблений каждого слова словаря: $X[i,j]$ - количество слов с индексом $j$ в документе $i$.


![image](bag_of_words.png)



In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [11]:
print(X_train_counts.shape)

(11314, 130107)


In [12]:
count_vect.vocabulary_.get('get') # индекс слова в словаре

58830

Данные представляют собой разряженную матрицу (sparse matrix) - матрицу с большим количеством нулей:

In [13]:
X_train_counts

<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

## TF-IDF

Ещё один способ работы с текстовыми данными — [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) (**T**erm **F**requency–**I**nverse **D**ocument **F**requency). Рассмотрим коллекцию текстов $D$.  Для каждого уникального слова $t$ из документа $d \in D$ вычислим следующие величины:

1. Term Frequency – количество вхождений слова в отношении к общему числу слов в тексте:
$$\text{tf}(t, d) = \frac{n_{td}}{\sum_{t \in d} n_{td}},$$
где $n_{td}$ — количество вхождений слова $t$ в текст $d$.
1. Inverse Document Frequency
$$\text{idf}(t, D) = \log \frac{\left| D \right|}{\left| \{d\in D: t \in d\} \right|},$$
где $\left| \{d\in D: t \in d\} \right|$ – количество текстов в коллекции, содержащих слово $t$.

Тогда для каждой пары (слово, текст) $(t, d)$ вычислим величину:
$$\text{tf-idf}(t,d, D) = \text{tf}(t, d)\cdot \text{idf}(t, D).$$

Отметим, что значение $\text{tf}(t, d)$ корректируется для часто встречающихся общеупотребимых слов при помощи значения $\text{idf}(t, D).$

Признаковым описанием одного объекта $d \in D$ будет вектор $\bigg(\text{tf-idf}(t,d, D)\bigg)_{t\in V}$, где $V$ – словарь всех слов, встречающихся в коллекции $D$.

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(11314, 130107)
